<a href="https://colab.research.google.com/github/danielroth12/Multilevel-Monte-Carlo-learning/blob/main/multi_individual_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Multilevel algorithm using 8 networks
#For more detailed explanations of the training and model parameters
#see Gerstner et al. "Multilevel Monte Carlo learning." arXiv preprint arXiv:2102.08734 (2021).

#Packages
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import time
from tensorflow.python.ops import init_ops
from tensorflow.contrib.layers.python.layers import initializers
from tensorflow.python.training.moving_averages import assign_moving_average
%tensorflow_version 1.x

#Basic network framework according to Beck, Christian, et al. "Solving the Kolmogorov PDE by means of deep learning." Journal of Scientific Computing 88.3 (2021): 1-28.
#The individual definition was repeated for each of the 8 networks
def neural_net(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)

  def _batch_normalization(_x):
    beta = tf.get_variable('beta', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x
def neural_net_p1_p0(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)
  def _batch_normalization(_x):
    beta = tf.get_variable('beta2', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma2', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean2', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance2', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments2')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights2',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer2_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x  
def neural_net_p2_p1(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  
  
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)
  def _batch_normalization(_x):
    beta = tf.get_variable('beta3', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma3', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean3', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance3', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments3')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights3',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer3_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x   
def neural_net_p3_p2(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)
  def _batch_normalization(_x):
    beta = tf.get_variable('beta4', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma4', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean4', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance4', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments4')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights4',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer4_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x     
def neural_net_p4_p3(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)
  def _batch_normalization(_x):
    beta = tf.get_variable('beta5', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma5', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean5', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance5', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments5')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights5',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer5_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x       
def neural_net_p5_p4(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)
  def _batch_normalization(_x):
    beta = tf.get_variable('beta6', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma6', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean6', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance6', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments6')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights6',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer6_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x    
def neural_net_p6_p5(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)

  def _batch_normalization(_x):
    beta = tf.get_variable('beta7', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma7', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean7', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance7', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments7')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights7',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer7_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x         

def neural_net_p7_p6(x, xi_approx, neurons, is_training, name,
                    mv_decay=0.9, dtype=tf.float32):
  
  
  def approx_test(): return xi_approx
  def approx_learn(): return x
  x = tf.cond(is_training, approx_learn,approx_test)

  def _batch_normalization(_x):
    beta = tf.get_variable('beta8', [_x.get_shape()[-1]],
                           dtype, init_ops.zeros_initializer())
    gamma = tf.get_variable('gamma8', [_x.get_shape()[-1]],
                            dtype, init_ops.ones_initializer())
    mv_mean = tf.get_variable('mv_mean8', [_x.get_shape()[-1]],
                              dtype, init_ops.zeros_initializer(),
                              trainable=False)
    mv_variance = tf.get_variable('mv_variance8', [_x.get_shape()[-1]],
                                  dtype, init_ops.ones_initializer(),
                                  trainable=False)
    mean, variance = tf.nn.moments(_x, [0], name='moments8')
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_mean, mean,
                                               mv_decay, True))
    tf.add_to_collection(tf.GraphKeys.UPDATE_OPS,
                         assign_moving_average(mv_variance, variance,
                                               mv_decay, False))
    mean, variance = tf.cond(is_training,
                             lambda: (mean, variance),
                             lambda: (mv_mean, mv_variance))
    return tf.nn.batch_normalization(_x, mean, variance,
                                     beta, gamma, 1e-6)
  def _layer(_x, out_size, activation_fn):
    w = tf.get_variable('weights8',
                        [_x.get_shape().as_list()[-1], out_size],
                        dtype, initializers.xavier_initializer())
    return activation_fn(_batch_normalization(tf.matmul(_x, w)))
  with tf.variable_scope(name):
      x = _batch_normalization(x)
      for i in range(len(neurons)):
        with tf.variable_scope('layer8_%i_' % (i + 1)):
          x = _layer(x, neurons[i],
            tf.nn.tanh if i < len(neurons)-1 else tf.identity)
  return x         

#Basic network framework according to Beck, Christian, et al. "Solving the Kolmogorov PDE by means of deep learning." Journal of Scientific Computing 88.3 (2021): 1-28.
#Adjustments:
#- exponential decay (individual parameters possible)
#- each network trained individually but in one session (line 534)
#- overall error calculated in line 411

def train_and_test(xi, xi_approx, xi_p1_p0, xi_p7_p6, xi_p6_p5, xi_p5_p4, xi_p4_p3, xi_p3_p2, xi_p2_p1, x_sde, phi_p0, phi_p7_p6, phi_p6_p5, phi_p5_p4, phi_p4_p3, phi_p3_p2, phi_p2_p1, phi_p1_p0, u_reference, u_reference_p0,u_reference_p7_p6,u_reference_p6_p5,u_reference_p5_p4,u_reference_p4_p3,u_reference_p3_p2,u_reference_p2_p1,u_reference_p1_p0, neurons,
          train_steps,mc_rounds, mc_freq, file_name,
          dtype=tf.float32):
  def _approximate_errors():
    lr, gs, lr10,lr21,lr32,lr43,lr54,lr65 = sess.run([learning_rate, global_step,learning_rate_p1_p0,learning_rate_p2_p1,learning_rate_p3_p2,learning_rate_p4_p3,learning_rate_p5_p4,learning_rate_p6_p5])
    li_err, li_err_p1_p0, li_err_kombination, li_err_p2_p1, li_err_p3_p2,li_err_p4_p3,li_err_p5_p4,li_err_p6_p5,li_err_p7_p6 = 0., 0., 0.,0.,0.,0.,0.,0.,0.
    for _ in range(mc_rounds):
      li, appr, ref, li_p7_p6, li_p6_p5, li_p5_p4, li_p4_p3, li_p3_p2, li_p2_p1,li_p1_p0, li_kombination = sess.run([err_l_inf,  approx, reference, err_l_p7_p6, err_l_p6_p5, err_l_p5_p4, err_l_p4_p3, err_l_p3_p2, err_l_p2_p1,err_l_p1_p0, err_l_kombination],
                                              feed_dict={is_training: False})
      li_err, li_err_p1_p0, li_err_kombination, li_err_p2_p1, li_err_p3_p2, li_err_p4_p3, li_err_p5_p4, li_err_p6_p5, li_err_p7_p6= (np.maximum(li_err, li),np.maximum(li_err_p1_p0, li_p1_p0)
          ,np.maximum(li_kombination, li_err_kombination),np.maximum(li_err_p2_p1, li_p2_p1),np.maximum(li_err_p3_p2, li_p3_p2),np.maximum(li_err_p4_p3, li_p4_p3),np.maximum(li_err_p5_p4, li_p5_p4),np.maximum(li_err_p6_p5, li_p6_p5),np.maximum(li_err_p7_p6, li_p7_p6))
    t_mc = time.time()
    file_out.write('%i, %f, %f, %f, %f, %f, %f, '
                    ' %f, %f, %f, %f, %f, %f \n' % (gs, li_err_kombination, lr,
                      t1_train - t0_train, t_mc - t1_train, appr, ref, lr10,lr21,lr32,lr43,lr54,lr65))
    file_out.flush()
    
  t0_train = time.time()
  is_training = tf.placeholder(tf.bool, [])
  u_approx = neural_net(xi, xi_approx, neurons, is_training, 'u_approx', dtype=dtype)
  u_approx_p1_p0 = neural_net_p1_p0(xi_p1_p0, xi_approx, neurons, is_training, 'u_approx_p1_p0', dtype=dtype)
  u_approx_p2_p1 = neural_net_p2_p1(xi_p2_p1, xi_approx, neurons, is_training, 'u_approx_p2_p1', dtype=dtype)
  u_approx_p3_p2 = neural_net_p3_p2(xi_p3_p2, xi_approx, neurons, is_training, 'u_approx_p3_p2', dtype=dtype)
  u_approx_p4_p3 = neural_net_p4_p3(xi_p4_p3, xi_approx, neurons, is_training, 'u_approx_p4_p3', dtype=dtype)  
  u_approx_p5_p4 = neural_net_p5_p4(xi_p5_p4, xi_approx, neurons, is_training, 'u_approx_p5_p4', dtype=dtype)  
  u_approx_p6_p5 = neural_net_p6_p5(xi_p6_p5, xi_approx, neurons, is_training, 'u_approx_p6_p5', dtype=dtype)  
  u_approx_p7_p6 = neural_net_p7_p6(xi_p7_p6, xi_approx, neurons, is_training, 'u_approx_p7_p6', dtype=dtype)  
  
  loss_p0 = tf.reduce_mean(tf.squared_difference(u_approx, phi_p0))
  loss_p1_p0 = tf.reduce_mean(tf.squared_difference(u_approx_p1_p0, phi_p1_p0))
  loss_p2_p1 = tf.reduce_mean(tf.squared_difference(u_approx_p2_p1, phi_p2_p1))
  loss_p3_p2 = tf.reduce_mean(tf.squared_difference(u_approx_p3_p2, phi_p3_p2))
  loss_p4_p3 = tf.reduce_mean(tf.squared_difference(u_approx_p4_p3, phi_p4_p3))
  loss_p5_p4 = tf.reduce_mean(tf.squared_difference(u_approx_p5_p4, phi_p5_p4))
  loss_p6_p5 = tf.reduce_mean(tf.squared_difference(u_approx_p6_p5, phi_p6_p5))
  loss_p7_p6 = tf.reduce_mean(tf.squared_difference(u_approx_p7_p6, phi_p7_p6))
  
  approx=tf.reduce_mean(u_approx)
  approx_p1_p0=tf.reduce_mean(u_approx_p1_p0)
  reference=tf.reduce_mean(u_reference_p0)

  err = tf.abs(u_approx - u_reference_p0)
  err_p1_p0 = tf.abs(u_approx_p1_p0 - u_reference_p1_p0)
  err_p2_p1 = tf.abs(u_approx_p2_p1 - u_reference_p2_p1)
  err_p3_p2 = tf.abs(u_approx_p3_p2 - u_reference_p3_p2)
  err_p4_p3 = tf.abs(u_approx_p4_p3 - u_reference_p4_p3)
  err_p5_p4 = tf.abs(u_approx_p5_p4 - u_reference_p5_p4)
  err_p6_p5 = tf.abs(u_approx_p6_p5 - u_reference_p6_p5)
  err_p7_p6 = tf.abs(u_approx_p7_p6 - u_reference_p7_p6)

  err_kombination=tf.abs(u_approx   + u_approx_p7_p6 + u_approx_p6_p5 + u_approx_p5_p4 + u_approx_p4_p3 + u_approx_p3_p2 + u_approx_p2_p1 + u_approx_p1_p0 - u_reference)
  err_l_inf = tf.reduce_max(err)
  err_l_p2_p1 = tf.reduce_max(err_p2_p1)
  err_l_p3_p2 = tf.reduce_max(err_p3_p2)
  err_l_p4_p3 = tf.reduce_max(err_p4_p3)
  err_l_p5_p4 = tf.reduce_max(err_p5_p4)
  err_l_p6_p5 = tf.reduce_max(err_p6_p5)
  err_l_p7_p6 = tf.reduce_max(err_p7_p6)
  err_l_p1_p0 = tf.reduce_max(err_p1_p0)
  err_l_kombination = tf.reduce_max(err_kombination)

  lr = 0.01
  step_rate = 40000
  decay = 0.1
  global_step = tf.Variable(1, trainable=False)
  increment_global_step = tf.assign(global_step, global_step + 1)
  learning_rate = tf.train.exponential_decay(lr, global_step, step_rate, decay, staircase=True)
  optimizer = tf.train.AdamOptimizer(learning_rate)

  lr_p1_p0 = lr
  step_rate_p1_p0 = step_rate
  decay_p1_p0 = 0.1
  global_step_p1_p0 = tf.Variable(0, trainable=False)
  increment_global_step_p1_p0 = tf.assign(global_step_p1_p0, global_step_p1_p0 + 1)
  learning_rate_p1_p0 = tf.train.exponential_decay(lr_p1_p0, global_step_p1_p0, step_rate_p1_p0, decay_p1_p0, staircase=True)
  optimizer_p1_p0 = tf.train.AdamOptimizer(learning_rate_p1_p0)

  lr_p2_p1 = lr
  step_rate_p2_p1 =step_rate
  decay_p2_p1 = 0.1
  global_step_p2_p1 = tf.Variable(0, trainable=False)
  increment_global_step_p2_p1 = tf.assign(global_step_p2_p1, global_step_p2_p1 + 1)
  learning_rate_p2_p1 = tf.train.exponential_decay(lr_p2_p1, global_step_p2_p1, step_rate_p2_p1, decay_p2_p1, staircase=True)
  optimizer_p2_p1 = tf.train.AdamOptimizer(learning_rate_p2_p1)

  lr_p3_p2 = lr
  step_rate_p3_p2 = step_rate
  decay_p3_p2 = 0.1
  global_step_p3_p2 = tf.Variable(0, trainable=False)
  increment_global_step_p3_p2 = tf.assign(global_step_p3_p2, global_step_p3_p2 + 1)
  learning_rate_p3_p2 = tf.train.exponential_decay(lr_p3_p2, global_step_p3_p2, step_rate_p3_p2, decay_p3_p2, staircase=True)
  optimizer_p3_p2 = tf.train.AdamOptimizer(learning_rate_p3_p2)

  lr_p4_p3 = lr
  step_rate_p4_p3 = step_rate
  decay_p4_p3 = 0.1
  global_step_p4_p3 = tf.Variable(0, trainable=False)
  increment_global_step_p4_p3 = tf.assign(global_step_p4_p3, global_step_p4_p3 + 1)
  learning_rate_p4_p3 = tf.train.exponential_decay(lr_p4_p3, global_step_p4_p3, step_rate_p4_p3, decay_p4_p3, staircase=True)
  optimizer_p4_p3 = tf.train.AdamOptimizer(learning_rate_p4_p3)
  
  lr_p5_p4 = lr
  step_rate_p5_p4 = step_rate
  decay_p5_p4 = 0.1
  global_step_p5_p4 = tf.Variable(0, trainable=False)
  increment_global_step_p5_p4 = tf.assign(global_step_p5_p4, global_step_p5_p4 + 1)
  learning_rate_p5_p4 = tf.train.exponential_decay(lr_p5_p4, global_step_p5_p4, step_rate_p5_p4, decay_p5_p4, staircase=True)
  optimizer_p5_p4 = tf.train.AdamOptimizer(learning_rate_p5_p4)

  #p6_p5
  lr_p6_p5 = lr
  step_rate_p6_p5 = step_rate
  decay_p6_p5 = 0.1
  global_step_p6_p5 = tf.Variable(0, trainable=False)
  increment_global_step_p6_p5 = tf.assign(global_step_p6_p5, global_step_p6_p5 + 1)
  learning_rate_p6_p5 = tf.train.exponential_decay(lr_p6_p5, global_step_p6_p5, step_rate_p6_p5, decay_p6_p5, staircase=True)
  optimizer_p6_p5 = tf.train.AdamOptimizer(learning_rate_p6_p5)

  #p7_p6
  lr_p7_p6 = lr
  step_rate_p7_p6 =step_rate
  decay_p7_p6 = 0.1
  global_step_p7_p6 = tf.Variable(0, trainable=False)
  increment_global_step_p7_p6 = tf.assign(global_step_p7_p6, global_step_p7_p6 + 1)
  learning_rate_p7_p6 = tf.train.exponential_decay(lr_p7_p6, global_step_p7_p6, step_rate_p7_p6, decay_p7_p6, staircase=True)
  optimizer_p7_p6 = tf.train.AdamOptimizer(learning_rate_p7_p6)


  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx')
  with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss_p0, global_step)
  
  update_ops_p1_p0 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p1_p0')
  with tf.control_dependencies(update_ops_p1_p0):
    train_op_p1_p0 = optimizer_p1_p0.minimize(loss_p1_p0, global_step_p1_p0)  

  update_ops_p2_p1 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p2_p1')
  with tf.control_dependencies(update_ops_p2_p1):
    train_op_p2_p1 = optimizer_p2_p1.minimize(loss_p2_p1, global_step_p2_p1)   
    
  update_ops_p3_p2 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p3_p2')
  with tf.control_dependencies(update_ops_p3_p2):
    train_op_p3_p2 = optimizer_p3_p2.minimize(loss_p3_p2, global_step_p3_p2)   

  update_ops_p4_p3 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p4_p3')
  with tf.control_dependencies(update_ops_p4_p3):
    train_op_p4_p3 = optimizer_p4_p3.minimize(loss_p4_p3, global_step_p4_p3)    

  update_ops_p5_p4 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p5_p4')
  with tf.control_dependencies(update_ops_p5_p4):
    train_op_p5_p4 = optimizer_p5_p4.minimize(loss_p5_p4, global_step_p5_p4) 

  update_ops_p6_p5 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p6_p5')
  with tf.control_dependencies(update_ops_p6_p5):
    train_op_p6_p5 = optimizer_p6_p5.minimize(loss_p6_p5, global_step_p6_p5) 

  update_ops_p7_p6 = tf.get_collection(tf.GraphKeys.UPDATE_OPS, 'u_approx_p7_p6')
  with tf.control_dependencies(update_ops_p7_p6):
    train_op_p7_p6 = optimizer_p7_p6.minimize(loss_p7_p6, global_step_p7_p6) 
        
  file_out = open(file_name, 'w')
  file_out.write('step,li_err, learning_rate, time_train, time_mc ,approx, reference, lr10,lr21,lr32,lr43,lr54,lr65 \n ')
    

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(1,train_steps):
      if step % mc_freq == 0:
        print(step)
        t1_train = time.time()
        _approximate_errors()
        t0_train = time.time()      
      sess.run([train_op,train_op_p1_p0,train_op_p2_p1,train_op_p3_p2,train_op_p4_p3,train_op_p5_p4,train_op_p6_p5,train_op_p7_p6], feed_dict={is_training:True})
    t1_train = time.time()
    _approximate_errors()
  file_out.close()
  
#Model and training parameter specification    
for i in range(1,2):
 #print(i)
 tf.reset_default_graph()
 tf.random.set_random_seed(i)
 with tf.Session()  as sess:
  dtype = tf.float32

  #Set network and training parameter 
  batch_size      =1200000
  batch_size_p1_p0=64000 
  batch_size_p2_p1=32000
  batch_size_p3_p2=16000
  batch_size_p4_p3=8000
  batch_size_p5_p4=4000
  batch_size_p6_p5=2000
  batch_size_p7_p6=1000
  batch_size_approx=2000000 
  N, d = 1, 5
  N_p0=N
  N_p1_p0=N 
  N_p2_p1=N*2
  N_p3_p2=N*4
  N_p4_p3=N*8
  N_p5_p4=N*16
  N_p6_p5=N*32
  N_p7_p6=N*64
  neurons = [50, 50, 1]
  train_steps =150000
  Ksteps_p7_p6=11000
  Ksteps_p6_p5=13000
  Ksteps_p5_p4=14000
  Ksteps_p4_p3=15000
  Ksteps_p3_p2=18000
  Ksteps_p2_p1=19000
  Ksteps_p1_p0=20000
  Ksteps_p0=   train_steps    
  mc_rounds, mc_freq = 100, 5000
  mc_samples_ref_p0=1#1530596
  mc_samples_ref_p1_p0=1#5000
  mc_samples_ref_p2_p1=1#1000
  mc_samples_ref_p3_p2=1#500
  mc_samples_ref_p4_p3=1#500
  mc_samples_ref_p5_p4=1#100
  mc_samples_ref_p6_p5=1#100
  mc_samples_ref, mc_rounds_ref_p0, mc_rounds_ref_p1_p0 = 1, 1000000,1000000

  #Define training and test interval
  s_0_l=80.0
  s_0_r=120.0
  sigma_l=0.1
  sigma_r=0.2
  mu_l=0.02
  mu_r=0.05
  T_l=0.9
  T_r=1.0
  K_l=109.0
  K_r=110.0
  s_0_l_approx=80.4
  s_0_r_approx=119.6
  sigma_l_approx=0.11
  sigma_r_approx=0.19
  mu_l_approx=0.03
  mu_r_approx=0.04
  T_l_approx=0.91
  T_r_approx=0.99
  K_l_approx=109.1
  K_r_approx=109.9
  s0 = tf.random_uniform((batch_size,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)
  sigma=tf.random_uniform((batch_size,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu=tf.random_uniform((batch_size,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T=tf.random_uniform((batch_size,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K=tf.random_uniform((batch_size,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p1_p0 = tf.stack((tf.random_uniform((batch_size_p1_p0,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p1_p0=tf.random_uniform((batch_size_p1_p0,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p1_p0=tf.random_uniform((batch_size_p1_p0,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p1_p0=tf.random_uniform((batch_size_p1_p0,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p1_p0=tf.random_uniform((batch_size_p1_p0,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p2_p1 = tf.stack((tf.random_uniform((batch_size_p2_p1,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p2_p1=tf.random_uniform((batch_size_p2_p1,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p2_p1=tf.random_uniform((batch_size_p2_p1,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p2_p1=tf.random_uniform((batch_size_p2_p1,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p2_p1=tf.random_uniform((batch_size_p2_p1,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p3_p2 = tf.stack((tf.random_uniform((batch_size_p3_p2,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p3_p2=tf.random_uniform((batch_size_p3_p2,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p3_p2=tf.random_uniform((batch_size_p3_p2,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p3_p2=tf.random_uniform((batch_size_p3_p2,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p3_p2=tf.random_uniform((batch_size_p3_p2,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p4_p3 = tf.stack((tf.random_uniform((batch_size_p4_p3,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p4_p3=tf.random_uniform((batch_size_p4_p3,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p4_p3=tf.random_uniform((batch_size_p4_p3,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p4_p3=tf.random_uniform((batch_size_p4_p3,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p4_p3=tf.random_uniform((batch_size_p4_p3,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p5_p4 = tf.stack((tf.random_uniform((batch_size_p5_p4,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p5_p4=tf.random_uniform((batch_size_p5_p4,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p5_p4=tf.random_uniform((batch_size_p5_p4,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p5_p4=tf.random_uniform((batch_size_p5_p4,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p5_p4=tf.random_uniform((batch_size_p5_p4,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p6_p5 = tf.stack((tf.random_uniform((batch_size_p6_p5,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p6_p5=tf.random_uniform((batch_size_p6_p5,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p6_p5=tf.random_uniform((batch_size_p6_p5,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p6_p5=tf.random_uniform((batch_size_p6_p5,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p6_p5=tf.random_uniform((batch_size_p6_p5,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  s0_p7_p6 = tf.stack((tf.random_uniform((batch_size_p7_p6 ,1), minval=s_0_l,
                                 maxval=s_0_r, dtype=dtype)))
  sigma_p7_p6 =tf.random_uniform((batch_size_p7_p6 ,1),
                            minval=sigma_l,maxval=sigma_r, dtype=dtype)
  mu_p7_p6 =tf.random_uniform((batch_size_p7_p6 ,1),
                            minval=mu_l,maxval=mu_r, dtype=dtype)
  T_p7_p6 =tf.random_uniform((batch_size_p7_p6 ,1),
                            minval=T_l,maxval=T_r, dtype=dtype)
  K_p7_p6 =tf.random_uniform((batch_size_p7_p6 ,1),
                      minval=K_l,maxval=K_r, dtype=dtype)
  xi=tf.reshape(tf.stack([s0,sigma,mu,T,K], axis=2), (batch_size,d))
  xi_p1_p0=tf.reshape(tf.stack([s0_p1_p0,sigma_p1_p0,mu_p1_p0,T_p1_p0,K_p1_p0], axis=2), (batch_size_p1_p0,d))
  xi_p2_p1=tf.reshape(tf.stack([s0_p2_p1,sigma_p2_p1,mu_p2_p1,T_p2_p1,K_p2_p1], axis=2), (batch_size_p2_p1,d))
  xi_p3_p2=tf.reshape(tf.stack([s0_p3_p2,sigma_p3_p2,mu_p3_p2,T_p3_p2,K_p3_p2], axis=2), (batch_size_p3_p2,d))
  xi_p4_p3=tf.reshape(tf.stack([s0_p4_p3,sigma_p4_p3,mu_p4_p3,T_p4_p3,K_p4_p3], axis=2), (batch_size_p4_p3,d))
  xi_p5_p4=tf.reshape(tf.stack([s0_p5_p4,sigma_p5_p4,mu_p5_p4,T_p5_p4,K_p5_p4], axis=2), (batch_size_p5_p4,d))
  xi_p6_p5=tf.reshape(tf.stack([s0_p6_p5,sigma_p6_p5,mu_p6_p5,T_p6_p5,K_p6_p5], axis=2), (batch_size_p6_p5,d))
  xi_p7_p6=tf.reshape(tf.stack([s0_p7_p6,sigma_p7_p6,mu_p7_p6,T_p7_p6,K_p7_p6], axis=2), (batch_size_p7_p6,d))
  s0_approx = tf.random_uniform((batch_size_approx,  1 ), 
                            minval=s_0_l_approx,maxval=s_0_r_approx, dtype=dtype)
  sigma_approx=tf.random_uniform((batch_size_approx,  1 ), 
                            minval=sigma_l_approx,maxval=sigma_r_approx, dtype=dtype)
  mu_approx=tf.random_uniform((batch_size_approx,1),
                            minval=mu_l_approx,maxval=mu_r_approx, dtype=dtype)
  T_approx=tf.random_uniform((batch_size_approx,1),
                            minval=T_l_approx,maxval=T_r_approx, dtype=dtype)
  K_approx=tf.random_uniform((batch_size_approx,1),
                            minval=K_l_approx,maxval=K_r_approx, dtype=dtype)
  xi_approx=tf.reshape(tf.stack([s0_approx,sigma_approx,mu_approx,T_approx,K_approx], axis=2), (batch_size_approx,d))
  #Closed solution as reference
  tfd = tfp.distributions
  dist = tfd.Normal(loc=tf.cast(0.,tf.float32), scale=tf.cast(1.,tf.float32))
  d1=tf.math.divide(
  (tf.log(tf.math.divide(s0_approx,K_approx))+(mu_approx + 0.5*sigma_approx**2)*T_approx) , (sigma_approx*tf.sqrt(T_approx)))
  d2=tf.math.divide(
  (tf.log(tf.math.divide(s0_approx,K_approx))+(mu_approx - 0.5*sigma_approx**2)*T_approx) , (sigma_approx*tf.sqrt(T_approx)))
  u_reference= tf.multiply(s0_approx,(dist.cdf(d1)))-K_approx*tf.exp(-mu_approx*T_approx)*(dist.cdf(d2))

 #Multilevel Monte Carlo path simulation
 def sde_body_p1(idx, s,sigma,mu,T,K, samples): 
    z = tf.random_normal(shape=(samples, batch_size_p1_p0, 1),
                         stddev=1., dtype=dtype)
    s=s + mu *s * h_p1_p0 +sigma * s *tf.sqrt(h_p1_p0)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(h_p1_p0)*z)**2-h_p1_p0)  
    return tf.add(idx, 1), s, sigma,mu,T,K
 def mc_body_p1(idx, p):
    _, _x, sigma,mu,T,K = tf.while_loop(lambda _idx, s, sigma,mu,T,K: _idx < N,
                          lambda _idx, s, sigma,mu,T,K: sde_body_p1(_idx, s,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p1)

    return idx + 1, p + tf.reduce_mean(phi(_x,sigma,mu,T,K, 2), axis=0)
 def sde_body_p1_p0(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_p1_p0, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p1_p0, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p1_p0_coarse= T / (N_p1_p0)
    h_p1_p0_fine= T / (N_p1_p0*2)
    hfine=h_p1_p0_fine
    hcoarse=h_p1_p0_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine, sigma,mu,T,K   
 def mc_body_p1_p0(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p1_p0,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p1_p0(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p1_p0)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_p2_p1(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_p2_p1, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p2_p1, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p2_p1_coarse=T / (N_p2_p1)
    h_p2_p1_fine=T / (N_p2_p1*2)
    hfine=h_p2_p1_fine
    hcoarse=h_p2_p1_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine , sigma,mu,T,K
 def mc_body_p2_p1(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p2_p1,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p2_p1(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p2_p1)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_p3_p2(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_p3_p2, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p3_p2, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p3_p2_coarse=T / (N_p3_p2)
    h_p3_p2_fine=T / (N_p3_p2*2)
    hfine=h_p3_p2_fine
    hcoarse=h_p3_p2_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K
 def sde_body_p4_p3(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_p4_p3, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p4_p3, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p4_p3_coarse=T / (N_p4_p3)
    h_p4_p3_fine=T / (N_p4_p3*2)
    hfine=h_p4_p3_fine
    hcoarse=h_p4_p3_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K
 def sde_body_p5_p4(idx, s, sfine,sigma,mu,T,K, samples):
    z1 = tf.random_normal(shape=(samples, batch_size_p5_p4, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p5_p4, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p5_p4_coarse=T / (N_p5_p4)
    h_p5_p4_fine=T / (N_p5_p4*2)
    hfine=h_p5_p4_fine
    hcoarse=h_p5_p4_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K
 def sde_body_p6_p5(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_p6_p5, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p6_p5, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p6_p5_coarse=T / (N_p6_p5)
    h_p6_p5_fine=T / (N_p6_p5*2)
    hfine=h_p6_p5_fine
    hcoarse=h_p6_p5_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K
 def sde_body_p7_p6(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_p7_p6, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_p7_p6, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    h_p7_p6_coarse=T / (N_p7_p6)
    h_p7_p6_fine=T / (N_p7_p6*2)
    hfine=h_p7_p6_fine
    hcoarse=h_p7_p6_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K
 def mc_body_p3_p2(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p3_p2,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p3_p2(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p3_p2)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0)  
 def mc_body_p4_p3(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p4_p3,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p4_p3(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p4_p3)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def mc_body_p5_p4(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p5_p4,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p5_p4(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p5_p4)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def mc_body_p6_p5(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p6_p5,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p6_p5(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p6_p5)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def mc_body_p7_p6(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p7_p6,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p7_p6(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p7_p6)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def mc_body_p8_p7(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p8_p7,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_p8_p7(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p8_p7)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2), axis=0) -tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def mc_body_ref_p0(idx2, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p0,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p0(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p0),
                                                   loop_var_mc_ref_p0)
    return idx2 + 1, p + tf.reduce_mean(phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p0(idx, s, sfine,sigma,mu,T,K, samples): 
    z = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)

    hcoarse=h_p1_p0_coarse
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine , sigma,mu,T,K 
 def mc_body_ref_p1_p0(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p1_p0,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p1_p0(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p1_p0),
                                                   loop_var_mc_ref_p1_p0)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p1_p0(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    hfine=h_p1_p0_fine
    hcoarse=h_p1_p0_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine , sigma,mu,T,K     
 def mc_body_ref_p2_p1(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p2_p1,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p2_p1(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p2_p1),
                                                   loop_var_mc_ref_p2_p1)
    return idx + 1, p + tf.reduce_mean(phi(_xfine, sigma,mu,T,K,2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p2_p1(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    hfine=h_p2_p1_fine
    hcoarse=h_p2_p1_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K                           
 def mc_body_ref_p3_p2(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p3_p2,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p3_p2(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p3_p2),
                                                   loop_var_mc_ref_p3_p2)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p3_p2(idx, s, sfine,sigma,mu,T,K, samples):
    z1 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    hfine=h_p3_p2_fine
    hcoarse=h_p3_p2_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K                           
 def mc_body_ref_p4_p3(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p4_p3,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p4_p3(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p4_p3),
                                                   loop_var_mc_ref_p4_p3)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p4_p3(idx, s, sfine,sigma,mu,T,K, samples):
    z1 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    hfine=h_p4_p3_fine
    hcoarse=h_p4_p3_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K                           
 def mc_body_ref_p5_p4(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p5_p4,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p5_p4(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p5_p4),
                                                   loop_var_mc_ref_p5_p4)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p5_p4(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    hfine=h_p5_p4_fine
    hcoarse=h_p5_p4_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine    , sigma,mu,T,K                        
 def mc_body_ref_p6_p5(idx, p):
    _, _xcoarse, _xfine, sigma,mu,T,K  = tf.while_loop(lambda _idx, s, xfine, sigma,mu,T,K: _idx < N_p6_p5,
                          lambda _idx, s, xfine, sigma,mu,T,K: sde_body_ref_p6_p5(_idx, s, xfine,sigma,mu,T,K,
                                                   mc_samples_ref_p6_p5),
                                                   loop_var_mc_ref_p6_p5)
    return idx + 1, p + tf.reduce_mean(phi(_xfine,sigma,mu,T,K, 2)-phi(_xcoarse,sigma,mu,T,K, 2), axis=0) 
 def sde_body_ref_p6_p5(idx, s, sfine,sigma,mu,T,K, samples): 
    z1 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z2 = tf.random_normal(shape=(samples, batch_size_approx, 1),
                          stddev=1., dtype=dtype)
    z=(z1+z2)/tf.sqrt(2.)
    hfine=h_p6_p5_fine
    hcoarse=h_p6_p5_coarse
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z1 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z1)**2-hfine)
    sfine=sfine + mu *sfine * hfine +sigma * sfine *tf.sqrt(hfine)*z2 + 0.5 *sigma *sfine *sigma * ((tf.sqrt(hfine)*z2)**2-hfine)
    s=s + mu *s * hcoarse +sigma * s *tf.sqrt(hcoarse)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(hcoarse)*z)**2-hcoarse)    
    return tf.add(idx, 1), s, sfine  , sigma,mu,T,K    
 def phi(x,sigma,mu,T,K, axis=1):
    payoff=tf.exp(-mu * T)* tf.maximum(x - K, 0.)
    return payoff
 def sde_body_p0(idx, s,sigma,mu,T,K, samples): 
    z = tf.random_normal(shape=(samples, batch_size, 1),
                         stddev=1., dtype=dtype)
    h=T/N
    s=s + mu *s * h +sigma * s *tf.sqrt(h)*z + 0.5 *sigma *s *sigma * ((tf.sqrt(h)*z)**2-h)
    return tf.add(idx, 1), s, sigma,mu,T,K
 def mc_body_p0(idx, p):
    _, _x, _sigma,_mu,_T,_K = tf.while_loop(lambda _idx, s, sigma,mu,T,K: _idx < N,
                          lambda _idx, s, sigma,mu,T,K: sde_body_p0(_idx, s, sigma,mu,T,K,
                                                   mc_samples_ref),
                                                   loop_var_mc_p0)
    return idx + 1, p + tf.reduce_mean(phi(_x,_sigma,_mu,_T,_K, 2), axis=0)  
 loop_var_mc_p0 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size, 1), dtype) * s0, tf.ones((mc_samples_ref,batch_size, 1), dtype) * sigma,tf.ones((mc_samples_ref,batch_size, 1), dtype) * mu,tf.ones((mc_samples_ref,batch_size, 1), dtype) * T,tf.ones((mc_samples_ref,batch_size, 1), dtype) * K)
 loop_var_mc_p1_p0 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p1_p0, 1), dtype) * s0_p1_p0,tf.ones((mc_samples_ref,batch_size_p1_p0, 1), dtype) * s0_p1_p0, tf.ones((mc_samples_ref,batch_size_p1_p0, 1), dtype) * sigma_p1_p0,tf.ones((mc_samples_ref,batch_size_p1_p0, 1), dtype) * mu_p1_p0,tf.ones((mc_samples_ref,batch_size_p1_p0, 1), dtype) * T_p1_p0,tf.ones((mc_samples_ref,batch_size_p1_p0, 1), dtype) * K_p1_p0)
 loop_var_mc_p2_p1 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p2_p1, 1), dtype) * s0_p2_p1,tf.ones((mc_samples_ref,batch_size_p2_p1, 1), dtype) * s0_p2_p1, tf.ones((mc_samples_ref,batch_size_p2_p1, 1), dtype) * sigma_p2_p1,tf.ones((mc_samples_ref,batch_size_p2_p1, 1), dtype) * mu_p2_p1,tf.ones((mc_samples_ref,batch_size_p2_p1, 1), dtype) * T_p2_p1,tf.ones((mc_samples_ref,batch_size_p2_p1, 1), dtype) * K_p2_p1)
 loop_var_mc_p3_p2 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p3_p2, 1), dtype) * s0_p3_p2,tf.ones((mc_samples_ref,batch_size_p3_p2, 1), dtype) * s0_p3_p2, tf.ones((mc_samples_ref,batch_size_p3_p2, 1), dtype) * sigma_p3_p2,tf.ones((mc_samples_ref,batch_size_p3_p2, 1), dtype) * mu_p3_p2,tf.ones((mc_samples_ref,batch_size_p3_p2, 1), dtype) * T_p3_p2,tf.ones((mc_samples_ref,batch_size_p3_p2, 1), dtype) * K_p3_p2)
 loop_var_mc_p4_p3 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p4_p3, 1), dtype) * s0_p4_p3,tf.ones((mc_samples_ref,batch_size_p4_p3, 1), dtype) * s0_p4_p3, tf.ones((mc_samples_ref,batch_size_p4_p3, 1), dtype) * sigma_p4_p3,tf.ones((mc_samples_ref,batch_size_p4_p3, 1), dtype) * mu_p4_p3,tf.ones((mc_samples_ref,batch_size_p4_p3, 1), dtype) * T_p4_p3,tf.ones((mc_samples_ref,batch_size_p4_p3, 1), dtype) * K_p4_p3)
 loop_var_mc_p5_p4 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p5_p4, 1), dtype) * s0_p5_p4,tf.ones((mc_samples_ref,batch_size_p5_p4, 1), dtype) * s0_p5_p4, tf.ones((mc_samples_ref,batch_size_p5_p4, 1), dtype) * sigma_p5_p4,tf.ones((mc_samples_ref,batch_size_p5_p4, 1), dtype) * mu_p5_p4,tf.ones((mc_samples_ref,batch_size_p5_p4, 1), dtype) * T_p5_p4,tf.ones((mc_samples_ref,batch_size_p5_p4, 1), dtype) * K_p5_p4)
 loop_var_mc_p6_p5 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p6_p5, 1), dtype) * s0_p6_p5,tf.ones((mc_samples_ref,batch_size_p6_p5, 1), dtype) * s0_p6_p5, tf.ones((mc_samples_ref,batch_size_p6_p5, 1), dtype) * sigma_p6_p5,tf.ones((mc_samples_ref,batch_size_p6_p5, 1), dtype) * mu_p6_p5,tf.ones((mc_samples_ref,batch_size_p6_p5, 1), dtype) * T_p6_p5,tf.ones((mc_samples_ref,batch_size_p6_p5, 1), dtype) * K_p6_p5)
 loop_var_mc_p7_p6 = (tf.constant(0),tf.ones((mc_samples_ref,batch_size_p7_p6, 1), dtype) * s0_p7_p6,tf.ones((mc_samples_ref,batch_size_p7_p6, 1), dtype) * s0_p7_p6, tf.ones((mc_samples_ref,batch_size_p7_p6, 1), dtype) * sigma_p7_p6,tf.ones((mc_samples_ref,batch_size_p7_p6, 1), dtype) * mu_p7_p6,tf.ones((mc_samples_ref,batch_size_p7_p6, 1), dtype) * T_p7_p6,tf.ones((mc_samples_ref,batch_size_p7_p6, 1), dtype) * K_p7_p6)
 _, x_sde,sigma,mu,T,K = tf.while_loop(lambda idx, s, sigma,mu,T,K: idx < N,
                         lambda idx, s, sigma,mu,T,K: sde_body_p0(idx, s,sigma,mu,T,K, 1),
                         loop_var_mc_p0)
 _, u_p0  = tf.while_loop(lambda idx, p: idx < 1, mc_body_p0,
                      (tf.constant(0), tf.zeros((batch_size, 1), dtype)))
 u_mc_p0 = u_p0 / tf.cast(1, tf.float32)
 _, u_p1_p0 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p1_p0,
                      (tf.constant(0), tf.zeros((batch_size_p1_p0, 1), dtype)))
 u_mc_p1_p0 = u_p1_p0 / tf.cast(1, tf.float32)
 _, u_p2_p1 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p2_p1,
                      (tf.constant(0), tf.zeros((batch_size_p2_p1, 1), dtype)))
 u_mc_p2_p1 = u_p2_p1 / tf.cast(1, tf.float32)
 _, u_p3_p2 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p3_p2,
                      (tf.constant(0), tf.zeros((batch_size_p3_p2, 1), dtype)))
 u_mc_p3_p2 = u_p3_p2 / tf.cast(1, tf.float32)
 _, u_p4_p3 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p4_p3,
                      (tf.constant(0), tf.zeros((batch_size_p4_p3, 1), dtype)))
 u_mc_p4_p3 = u_p4_p3 / tf.cast(1, tf.float32)
 _, u_p5_p4 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p5_p4,
                      (tf.constant(0), tf.zeros((batch_size_p5_p4, 1), dtype)))
 u_mc_p5_p4 = u_p5_p4 / tf.cast(1, tf.float32)
 _, u_p6_p5 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p6_p5,
                      (tf.constant(0), tf.zeros((batch_size_p6_p5, 1), dtype)))
 u_mc_p6_p5 = u_p6_p5 / tf.cast(1, tf.float32)
 _, u_p7_p6 = tf.while_loop(lambda idx, p: idx < 1, mc_body_p7_p6,
                      (tf.constant(0), tf.zeros((batch_size_p7_p6, 1), dtype)))
 u_mc_p7_p6 = u_p7_p6 / tf.cast(1, tf.float32)
 N_approx=1
 N_approx_ref_p0=1
 u_reference_p0   =tf.multiply(s0_approx,(dist.cdf(d1)))-K_approx*tf.exp(-mu_approx*T_approx)*(dist.cdf(d2))
 u_reference_p1_p0=xi_approx*0.
 u_reference_p2_p1=xi_approx*0.
 u_reference_p3_p2=xi_approx*0.
 u_reference_p4_p3=xi_approx*0.
 u_reference_p5_p4=xi_approx*0.
 u_reference_p6_p5=xi_approx*0.
 u_reference_p7_p6=xi_approx*0.

 #Start training and testing                         
 train_and_test(xi, xi_approx, xi_p1_p0, xi_p7_p6, xi_p6_p5, xi_p5_p4, xi_p4_p3, xi_p3_p2,xi_p2_p1, tf.squeeze(tf.reduce_mean(x_sde,0,keepdims=True), axis=0), u_mc_p0, u_mc_p7_p6, u_mc_p6_p5, u_mc_p5_p4, u_mc_p4_p3, u_mc_p3_p2, u_mc_p2_p1, u_mc_p1_p0, u_reference, u_reference_p0,u_reference_p7_p6,u_reference_p6_p5,u_reference_p5_p4,u_reference_p4_p3,u_reference_p3_p2,u_reference_p2_p1, u_reference_p1_p0,
                          neurons, train_steps,
                          mc_rounds, mc_freq, 'multi-individual-trainsteps.csv', dtype)                      

TensorFlow 1.x selected.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
15
